# Techlabs: Matchmaking Process

Criteria for Match-making:
- Age: people within a certain age range, such as 50-60 or 60-70, above 70.
- Interests and Hobbies: similar interests and hobbies, such as gardening, cooking, musik or hiking.
- Location: live in the same city or region.
- Personality: compatible personality traits, such as extroverted or introverted.


Necessary information & format of it as inputs for function
1 User need to provide info on: name, age, interests, location, personality, number of desired matches
-> remaining questions:
1. What do we want to measure and include in the matchmaking?
2. how to connect this algorithm with the app? how to execute it?
3. 

Possible further functionalities:
1. send_message(self, sender, recipient, message): This function would allow a user to send a message to another user. It could take three arguments: sender and recipient, which are User objects representing the sender and recipient of the message, respectively, and message, which is a string containing the message text.
2. block_user(self, user, other_user): This function would allow a user to block another user, preventing them from appearing in the user's matches list or receiving messages from the user. It could take two arguments: user and other_user, which are User objects representing the user who is blocking and the user who is being blocked, respectively.
3. report_abuse(self, user, other_user): This function would allow a user to report another user for abuse or inappropriate behavior. It could take two arguments: user and other_user, which are User objects representing the user who is reporting the abuse and the user who is being reported, respectively.
4. set_location(self, user, location): This function would allow a user to update their location, which could be used to match them with other users who are nearby. It could take two arguments: user, which is a User object representing the user who is updating their location, and location, which is a string containing the user's new location.

In [63]:
class User:
    # Class to create a new user
    def __init__(self, name, age, interests, city, personality, num_matches):
        self.name = name
        self.age = age
        self.interests = interests
        self.city = city
        self.personality = personality
        self.num_matches = num_matches

class MatchMaker:
    # Class that creates matches
    def __init__(self, users):
        self.users = users

    def match_making(self, user):
        # Initialize empty list to store matches
        matches = []
       
        # Initialize dictionary to store the number of times each user has been suggested as a match
        match_counts = {u: 0 for u in self.users}

        # Loop through all users in the database
        for other_user in self.users:
            # Skip the current user
            if other_user == user:
                continue
            # Skip other_user if they have already been suggested the maximum number of times
            if match_counts[other_user] >= user.num_matches:
                continue

            # Calculate match score:
            # 1. Age
            age_categories = [(55, 65), (65, 70), (70, float("inf"))]
            for (min_age, max_age) in age_categories:
                if min_age <= user.age <= max_age and min_age <= other_user.age <= max_age:
                    age_score = 1
                    break
                else:
                    age_score = 0
            # 2. Common Interest 
            common_interests = len(set(user.interests) & set(other_user.interests))
            
            # 3. Same City
            if user.city == other_user.city:
                same_city = 1
            else:
                same_city = 0
            
            
            # Calculate score by addition
            match_score = age_score + common_interests + same_city

            # If match score is high enough, add other_user to the matches list
            if match_score > 1:
                matches.append((other_user, match_score))
                match_counts[other_user] += 1 
                
        # Sort the matches list by score in descending order
        matches.sort(key=lambda x: x[1], reverse=True)

        # Return the top N matches
        return [match[0] for match in matches[:user.num_matches]]
    
    def remove_match(self, user, other_user):
        # Find the current matches for user
        matches = self.match_making(user)

        # If other_user is in the matches list, remove it
        if other_user in matches:
            matches.remove(other_user)

        # Recalculate the matches for user to ensure that there are num_matches matches in the list
        while len(matches) < user.num_matches:
            # Find additional matches for user
            new_matches = self.match_making(user)
            # Add the new matches to the list
            matches.extend(new_matches)
        # Return the updated list of matches for user
        return matches

    
    def delete_account(self, user):
        # Remove user from the users list
        self.users.remove(user)
        
        # Loop through all users in the users list
        for other_user in self.users:
            # Remove user from other_user's matches list
            self.remove_match(other_user, user)
    
    def get_overview(self, user):
        # Initialize empty dictionary to store overview data
        overview = {'matches': [],'appeared_in_top_matches_of': 0,}

        # Find the current matches for user
        matches = self.match_making(user)
        # Add the user names of the matches to the overview data
        overview['matches'] = [match.name for match in matches]

        # Loop through all users in the users list
        for other_user in self.users:
            # Skip the current user
            if other_user == user:
                continue

            # Find the top matches for other_user
            top_matches = self.match_making(other_user)
            # Increment the appeared_in_top_matches_of counter if user is in the top matches for other_user
            if user in top_matches:
                overview['appeared_in_top_matches_of'] += 1

        return overview

In [64]:
# Create some users
user1 = User("Alice", 35, ["hiking", "cooking"], "New York", "outgoing", 3)
user2 = User("Bob", 60, ["gardening", "movies"], "Chicago", "introverted", 2)
user3 = User("Charlie", 45, ["hiking", "movies"], "New York", "outgoing", 2)
user4 = User("David", 55, ["gardening", "movies"], "Chicago", "introverted", 3)
user5 = User("Eve", 70, ["hiking", "cooking"], "New York", "outgoing", 1)

users = [user1, user2, user3, user4, user5]

# Create a match maker
match_maker = MatchMaker(users)

# Find the matches for user1
matches = match_maker.match_making(user1)
# Print the names of the matches
print([match.name for match in matches])
# Output: ['Charlie', 'Eve', 'Bob']

# Remove user2 from user1's matches list
match_maker.remove_match(user1, user2)

# Find the updated matches for user1
matches = match_maker.match_making(user1)
# Print the names of the matches
print([match.name for match in matches])
# Output: ['Charlie', 'Eve', 'David']

# Delete user1's account
match_maker.delete_account(user1)

# Find the matches for user2
matches = match_maker.match_making(user2)
# Print the names of the matches
print([match.name for match in matches])
# Output: ['David']

# Get the overview data for user2
overview = match_maker.get_overview(user2)
# Print the overview data
print(overview)
# Output: {'matches': ['David'], 'appeared_in_top_matches_of': 0}

['Eve', 'Charlie']
['Eve', 'Charlie']
['David']
{'matches': ['David'], 'appeared_in_top_matches_of': 1}
